In [90]:
import ee
import geemap
from geemap import chart
import math

ee.Initialize()
Map = geemap.Map()

<h3> Imports </h3>
<h4> Regions, Satellites, Visiblity Params </h4>

In [43]:
focus = ee.Geometry.Polygon(
        [[[27.350233348102517, -7.518171474050515],
          [27.350233348102517, -7.57841301205225],
          [27.436407359332986, -7.57841301205225],
          [27.436407359332986, -7.518171474050515]]], None, False)

modis = ee.ImageCollection("MODIS/006/MCD43A4")
ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

modisVis = {"min":0,"max":4000,"gamma":1.4}
rgbVis = {
    'min': 8097.96, 
    'max': 12128.04,
    'bands': ['SR_B4', 'SR_B3', 'SR_B2']
}

rgbVisS2 = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

EasternMine = ee.Geometry.Polygon(
        [[[27.37513382750328, -7.523187286499756],
          [27.37513382750328, -7.550926343072451],
          [27.41959412413414, -7.550926343072451],
          [27.41959412413414, -7.523187286499756]]], None, False)

s2 = ee.ImageCollection("COPERNICUS/S2_SR") 



<h4> Classification Points </h4>

In [44]:
water = ee.FeatureCollection([])
vegetation = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([27.376517541901784, -7.543591078372139]),
            {
              "landcover": 2,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([27.418746240632252, -7.539506842444226]),
            {
              "landcover": 2,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([27.36587453653069, -7.552780468215639]),
            {
              "landcover": 2,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([27.373770959870534, -7.569967505969657]),
            {
              "landcover": 2,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([27.389392145173268, -7.5590767891937025]),
            {
              "landcover": 2,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([27.428359277741627, -7.523680064445619]),
            {
              "landcover": 2,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([27.375659235017018, -7.522999329851055]),
            {
              "landcover": 2,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([27.38836217691155, -7.531026599579952]),
            {
              "landcover": 2,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([27.356089838044362, -7.564891220186839]),
            {
              "landcover": 2,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([27.403640039460377, -7.56931554740874]),
            {
              "landcover": 2,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([27.35385824014397, -7.522857852300823]),
            {
              "landcover": 2,
              "system:index": "10"
            })])
bare = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([27.401675991909222, -7.540000356472741]),
            {
              "landcover": 0,
              "system:index": "0"
            })])

In [45]:
Map.setCenter(27.3954, -7.55, 13) #Kanunka Village

area = focus.area().divide(1e6)
#print(area)

roi2000 = modis \
.filter(ee.Filter.date('2001-01-01', '2002-01-01')) \
.filter(ee.Filter.bounds(focus)) \
.select('Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band4', 'Nadir_Reflectance_Band3') \
.median() \
.clip(focus)

Map.addLayer(roi2000, modisVis, '2000')

train = ee.FeatureCollection(bare.merge(water).merge(vegetation))

training = roi2000.sampleRegions(
  collection=train,
  properties=['landcover'],
  scale=10
)

classifier = ee.Classifier.smileRandomForest(50).train(
  features= training,
  classProperty='landcover',
  inputProperties=roi2000.bandNames()
)

classified2000 = roi2000.classify(classifier)
Map.addLayer(classified2000, {'min': 0, 'max': 2, 'palette': ['red', 'blue', 'green']}, '2000_classified')

# Calculate Area by Class
areaImage = ee.Image.pixelArea().divide(1e6).addBands(classified2000)

# Using a Grouped Reducer
areas = areaImage.reduceRegion(
    reducer=ee.Reducer.sum().group(
        groupField=1,
        groupName='classification',
    ),
    geometry=focus,
    scale=100,
    tileScale=4,
    maxPixels=1e10
    )
classAreas = ee.List(areas.get('groups'))

vegetation2000 = ee.Number(ee.Dictionary(classAreas.get(1)).get('sum'))
bare2000 = ee.Dictionary(classAreas.get(0)).get('sum')

east_classified = classified2000.clip(EasternMine)
areaImageEast = ee.Image.pixelArea().divide(1e6).addBands(east_classified)

# Using a Grouped Reducer
areasEast = areaImageEast.reduceRegion(
    reducer=ee.Reducer.sum().group(
        groupField=1,
        groupName='classification',
    ),
    geometry=EasternMine,
    scale=100,
    tileScale=4,
    maxPixels=1e10
    )
classAreasEast = ee.List(areasEast.get('groups'))

vegetation2000east = ee.Number(ee.Dictionary(classAreasEast.get(1)).get('sum'))
print(vegetation2000east.getInfo())
Map.addLayer(east_classified, {'min': 0, 'max': 2, 'palette': ['red', 'blue', 'green']}, '2000_classified')
#print("Vegetation Area in 2000: ", vegetation2000)
#print("Bare Area in 2000: ", bare2000)

Map


14.02731981832111


Map(center=[-7.55, 27.3954], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [46]:
roi2020 = modis \
.filter(ee.Filter.date('2020-01-01', '2021-01-01')) \
.filter(ee.Filter.bounds(focus)) \
.select('Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band4', 'Nadir_Reflectance_Band3') \
.median() \
.clip(focus)

Map.addLayer(roi2020, modisVis, '2020')

classified2020 = roi2020.classify(classifier)
Map.addLayer(classified2020, {'min': 0, 'max': 2, 'palette': ['red', 'blue', 'green']}, '2020_classified')
# Calculate Area by Class
areaImage = ee.Image.pixelArea().divide(1e6).addBands(classified2020)

# Using a Grouped Reducer
areas = areaImage.reduceRegion(
      reducer=ee.Reducer.sum().group(
      groupField=1,
      groupName='classification',
    ),
    geometry=focus,
    scale=100,
    tileScale=4,
    maxPixels=1e10
    )
classAreas = ee.List(areas.get('groups'))
vegetation2020 = ee.Number(ee.Dictionary(classAreas.get(1)).get('sum'))
bare2020 = ee.Dictionary(classAreas.get(0)).get('sum')
print("Vegetation Area in 2020: " + str(vegetation2020.getInfo()))
#print("Bare Area in 2020: ", vegetation2020)

percentageLoss = (vegetation2000.subtract(vegetation2020)).divide(vegetation2000)
#print("Percentage of Vegetation Lost: ", percentageLoss)


Map

Vegetation Area in 2020: 43.06746906188297


Map(center=[-7.55, 27.3954], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

<h5> Dividing the Kanunka Region into Squares </h5>

In [84]:
def divide_geometry(s_length):
    # regionCoordinates = geometry.coordinates()
    regions = []
    i = 27.35
    while i <= 27.44:
        j = -7.52
        while j >= -7.58:
            region = ee.Geometry.Polygon(
                [[[i, j],
                  [i, j-s_length],
                  [i+s_length, j-s_length],
                  [i+s_length, j]]])
            regions.append((region))
            j = j-s_length
        i = i+s_length
    return regions



In [100]:
"""
Segment the given geometry into squares of given size (in km)
:param geometry: rectangle form geometry object
:return: list including all squares
"""
def create_segments(geometry, size):
    segments = []
    r_earth, dy, dx, pi = ee.Number(6378), ee.Number(size), ee.Number(size), ee.Number(math.pi)
    
    coords = ee.List(geometry.coordinates().get(0)).slice(0, -1)
    
    top = ee.Number(ee.List(coords.get(2)).get(1))
    left = ee.Number(ee.List(coords.get(0)).get(0))
    
    width = int(ee.Geometry.Point(coords.get(0)).distance(ee.Geometry.Point(coords.get(1))).divide(1000 * size).getInfo())
    height = int(ee.Geometry.Point(coords.get(1)).distance(ee.Geometry.Point(coords.get(2))).divide(1000 * size).getInfo())

    for y in range(height + 1):
        left = ee.Number(ee.List(coords.get(0)).get(0))
        for x in range(width + 1):
            #
            first = top
            second = dx.divide(r_earth)
            third = ee.Number(180).divide(pi)
            con = pi.divide(ee.Number(180))
            fourth = left.multiply(con).multiply(con).cos()
            
            new_lon = first.subtract(second.multiply(third).divide(fourth))
            #new_lon = top - (dx / r_earth) * (180 / pi) / math.cos(math.radians(left * pi/180))
            #new_lat = left  + (dy / r_earth) * (180 / pi)
            new_lat = left.add((dy.divide(r_earth)).multiply((ee.Number(180).divide(pi))))
            
            square = ee.Geometry.Polygon(
                [[[left, new_lon],
                  [new_lat, new_lon],
                  [new_lat, top],
                  [left, top]]], None, False)
            
            segments.append(square)
            
            left = new_lat
        top = new_lon
        
    return segments

In [108]:
regions = create_segments(focus, 1)
print(len(regions))

70


<h3> Evaluating Regions by Percent Vegetation Loss </h3>

In [141]:
def clipRegion2000(region):
    image2000 = modis \
            .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
            .filter(ee.Filter.bounds(region)) \
            .select(['Nadir_Reflectance_Band1',
                    'Nadir_Reflectance_Band4',
                    'Nadir_Reflectance_Band3']) \
            .median().clip(region)
    
    
    
    image.classify(classifier)
    
    
    
    
    
    
    return image

In [142]:
def clipRegion2020(region):
    image = modis \
            .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
            .filter(ee.Filter.bounds(region)) \
            .select(['Nadir_Reflectance_Band1',
                    'Nadir_Reflectance_Band4',
                    'Nadir_Reflectance_Band3']) \
            .median().clip(region)
    return image

In [143]:
def classifyMap(image):
    return image.classify(classifier)

In [145]:
def calculateVegArea(classified):
    veg1 = classified.eq(2)
    veg1Image = veg1.multiply(ee.Image.pixelArea())
    area1 = veg1Image.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': region,
      'scale': 500,
      'maxPixels': 1e10
      })
    veg1SqKm = ee.Number(area1.get('classification')).divide(1e6).getInfo()
    return veg1SqKm

In [146]:
def eval_change(vegArea_2000, vegArea_2020):
    if (vegArea_2000 == 0) and (vegArea_2020 == 0):
            print(str(num) + ': No Vegetation at Start or End')
            return 
    elif vegArea_2000 > 0:
        #percent decrease of vegetation area = ((original - new) / (original)) * 100
        percentDecrease = ((vegArea_2000-vegArea_2020)/vegArea_2000)*100
        print(str(num) + ': ' + str(percentDecrease) + '% Vegetation Loss')
        if percentDecrease >= 20:
    else:
        print(str(num) + ': Vegetation Increased from 0 Initially')

In [175]:
def eval_region(region):
    image2000 = modis \
            .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
            .filter(ee.Filter.bounds(region)) \
            .select(['Nadir_Reflectance_Band1',
                    'Nadir_Reflectance_Band4',
                    'Nadir_Reflectance_Band3']) \
            .median().clip(region)
    image2020 = modis \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.bounds(region)) \
            .select(['Nadir_Reflectance_Band1',
                    'Nadir_Reflectance_Band4',
                    'Nadir_Reflectance_Band3']) \
            .median().clip(region)
    #Classify the images
    classified2020 = image2020.classify(classifier)
    # Map.addLayer(classified2020, {'min': 1, 'max': 2, 'palette': ['red', 'green']}, str(num) + ' 2020_classified', False)

    classified2000 = image2000.classify(classifier)
    # Map.addLayer(classified2000, {'min': 1, 'max': 2, 'palette': ['red', 'green']},  str(num) + ' 2000_classified', False)

    veg1 = classified2000.eq(2)
    veg1Image = veg1.multiply(ee.Image.pixelArea())
    area1 = veg1Image.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': region,
      'scale': 500,
      'maxPixels': 1e10
      })
    veg1SqKm = ee.Number(area1.get('classification')).divide(1e6).getInfo()
    #print(str(num) + ' Area of Vegetation in 2000: ', veg1SqKm)

    veg2 = classified2020.eq(2)
    veg2Image = veg2.multiply(ee.Image.pixelArea())
    area2 = veg2Image.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': region,
      'scale': 500,
      'maxPixels': 1e10
      })
    
    veg2SqKm = ee.Number(area2.get('classification')).divide(1e6).getInfo()
    #print(str(num) + ' Area of Vegetation in 2020: ', veg2SqKm)
    
    percentDecrease = ((veg1SqKm.subtract(veg2SqKm)).divide(veg1SqKm)).multiply(100)
    
    return region.set('percentLoss', percentDecrease)
    # if (veg1SqKm == 0) and (veg2SqKm == 0):
    #     #print(str(num) + ': No Vegetation at Start or End')
    #     return region
    # elif veg1SqKm > 0:
    #     #percent decrease of vegetation area = ((original - new) / (original)) * 100
    #     percentDecrease = ((veg1SqKm-veg2SqKm)/veg1SqKm)*100
    #     #print(str(num) + ': ' + str(percentDecrease) + '% Vegetation Loss')
    #     if percentDecrease >= 20:
    #         return region
    # else:
    #     #print(str(num) + ': Vegetation Increased from 0 Initially')
    #     return None

In [182]:
def vegLoss_EX(regionList): 
    Map.centerObject(focus, 13)
    Map.addLayer(roi2000, modisVis, '2000', False)
    Map.addLayer(roi2020, modisVis, '2020', False)
    classified1 = roi2000.classify(classifier)
    classified2 = roi2020.classify(classifier)
    Map.addLayer(classified1, {'min': 1, 'max': 2, 'palette': ['red', 'green']}, '2000_classified', False)
    Map.addLayer(classified2, {'min': 1, 'max': 2, 'palette': ['red', 'green']}, '2020_classified', False)
    whole = s2 \
            .filter(ee.Filter.bounds(focus)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(focus)
    Map.addLayer(whole, rgbVisS2, 'whole', False)
    passed = []
    num = 0
    
    regions_passed = (ee.FeatureCollection(regionList))
    regions_passed = regions_passed.map(eval_region)
    #list(map(eval_region, regionList))
    
    #regions_passed = regions_passed.filter(ee.Filter.gte('percentLoss', 20))
    
    #passed.append(regions_passed)
    # regions_2000_clip = regionList.map(clipRegion2000)
    # regions_2020_clip = regionList.map(clipRegion2020)
    # classify_regions_2000 = classifyMap.map(regions_2000_clip)
    # classify_regions_2020 = classifyMap.map(regions_2020_clip)
    # region_vegArea_2000 = calculateVegArea.map(classify_regions_2000)
    # region_vegArea_2020 = calculateVegArea.map(classify_regions_2020)
    # regions_passed = list(map(eval_change, regionVegArea2000, regionVegArea2020))
    # passed = passed.append
#     for region in regionList:
#         image2000 = modis \
#             .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
#             .filter(ee.Filter.bounds(region)) \
#             .select(['Nadir_Reflectance_Band1',
#                     'Nadir_Reflectance_Band4',
#                     'Nadir_Reflectance_Band3']) \
#             .median().clip(region)
#         image2020 = modis \
#             .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
#             .filter(ee.Filter.bounds(region)) \
#             .select(['Nadir_Reflectance_Band1',
#                     'Nadir_Reflectance_Band4',
#                     'Nadir_Reflectance_Band3']) \
#             .median().clip(region)
#         # Classify the images
#         classified2020 = image2020.classify(classifier)
#         # Map.addLayer(classified2020, {'min': 1, 'max': 2, 'palette': ['red', 'green']}, str(num) + ' 2020_classified', False)

#         classified2000 = image2000.classify(classifier)
#         # Map.addLayer(classified2000, {'min': 1, 'max': 2, 'palette': ['red', 'green']},  str(num) + ' 2000_classified', False)

#         veg1 = classified2000.eq(2)
#         veg1Image = veg1.multiply(ee.Image.pixelArea())
#         area1 = veg1Image.reduceRegion(**{
#           'reducer': ee.Reducer.sum(),
#           'geometry': region,
#           'scale': 500,
#           'maxPixels': 1e10
#           })
#         veg1SqKm = ee.Number(area1.get('classification')).divide(1e6).getInfo()
#         #print(str(num) + ' Area of Vegetation in 2000: ', veg1SqKm)
        
#         veg2 = classified2020.eq(2)
#         veg2Image = veg2.multiply(ee.Image.pixelArea())
#         area2 = veg2Image.reduceRegion(**{
#           'reducer': ee.Reducer.sum(),
#           'geometry': region,
#           'scale': 500,
#           'maxPixels': 1e10
#           })
#         veg2SqKm = ee.Number(area2.get('classification')).divide(1e6).getInfo()
#         #print(str(num) + ' Area of Vegetation in 2020: ', veg2SqKm)
#         num = num+1
#         if (veg1SqKm == 0) and (veg2SqKm == 0):
#             passed.append(region)
#             print(str(num) + ': No Vegetation at Start or End')
#         elif veg1SqKm > 0:
#             #percent decrease of vegetation area = ((original - new) / (original)) * 100
#             percentDecrease = ((veg1SqKm-veg2SqKm)/veg1SqKm)*100
#             print(str(num) + ': ' + str(percentDecrease) + '% Vegetation Loss')
#             if percentDecrease >= 20:
#                 passed.append(region)
#         else:
#             print(str(num) + ': Vegetation Increased from 0 Initially')
    #print(passed)
    if len(passed) > 0:
        layer = ee.Geometry.MultiPolygon(passed)
        image = s2 \
            .filter(ee.Filter.bounds(layer)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(layer)
        Map.addLayer(image, rgbVisS2, 'vegLoss_ex_pass')
    Map
    return passed  

In [183]:
veg = vegLoss_EX(regions)
Map

EEException: A mapped function's arguments cannot be used in client-side operations

In [184]:
def vegLoss(regionList): 
    Map.centerObject(focus, 13)
    Map.addLayer(roi2000, modisVis, '2000', False)
    Map.addLayer(roi2020, modisVis, '2020', False)
    classified1 = roi2000.classify(classifier)
    classified2 = roi2020.classify(classifier)
    Map.addLayer(classified1, {'min': 1, 'max': 2, 'palette': ['red', 'green']}, '2000_classified', False)
    Map.addLayer(classified2, {'min': 1, 'max': 2, 'palette': ['red', 'green']}, '2020_classified', False)
    whole = s2 \
            .filter(ee.Filter.bounds(focus)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(focus)
    Map.addLayer(whole, rgbVisS2, 'whole', False)
    passed = []
    num = 0
    for region in regionList:
        image2000 = modis \
            .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
            .filter(ee.Filter.bounds(region)) \
            .select(['Nadir_Reflectance_Band1',
                    'Nadir_Reflectance_Band4',
                    'Nadir_Reflectance_Band3']) \
            .median().clip(region)
        image2020 = modis \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.bounds(region)) \
            .select(['Nadir_Reflectance_Band1',
                    'Nadir_Reflectance_Band4',
                    'Nadir_Reflectance_Band3']) \
            .median().clip(region)
        # Classify the images
        classified2020 = image2020.classify(classifier)
        # Map.addLayer(classified2020, {'min': 1, 'max': 2, 'palette': ['red', 'green']}, str(num) + ' 2020_classified', False)

        classified2000 = image2000.classify(classifier)
        # Map.addLayer(classified2000, {'min': 1, 'max': 2, 'palette': ['red', 'green']},  str(num) + ' 2000_classified', False)

        veg1 = classified2000.eq(2)
        veg1Image = veg1.multiply(ee.Image.pixelArea())
        area1 = veg1Image.reduceRegion(**{
          'reducer': ee.Reducer.sum(),
          'geometry': region,
          'scale': 500,
          'maxPixels': 1e10
          })
        veg1SqKm = ee.Number(area1.get('classification')).divide(1e6).getInfo()
        #print(str(num) + ' Area of Vegetation in 2000: ', veg1SqKm)
        
        veg2 = classified2020.eq(2)
        veg2Image = veg2.multiply(ee.Image.pixelArea())
        area2 = veg2Image.reduceRegion(**{
          'reducer': ee.Reducer.sum(),
          'geometry': region,
          'scale': 500,
          'maxPixels': 1e10
          })
        veg2SqKm = ee.Number(area2.get('classification')).divide(1e6).getInfo()
        #print(str(num) + ' Area of Vegetation in 2020: ', veg2SqKm)
        num = num+1
        if (veg1SqKm == 0) and (veg2SqKm == 0):
            passed.append(region)
            print(str(num) + ': No Vegetation at Start or End')
        elif veg1SqKm > 0:
            #percent decrease of vegetation area = ((original - new) / (original)) * 100
            percentDecrease = ((veg1SqKm-veg2SqKm)/veg1SqKm)*100
            print(str(num) + ': ' + str(percentDecrease) + '% Vegetation Loss')
            if percentDecrease >= 20:
                passed.append(region)
        else:
            print(str(num) + ': Vegetation Increased from 0 Initially')
        
    if len(passed) > 0:
        layer = ee.Geometry.MultiPolygon(passed)
        image = s2 \
            .filter(ee.Filter.bounds(layer)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(layer)
        Map.addLayer(image, rgbVisS2, 'vegLoss_pass')
    Map
    return passed  

In [185]:
passed_veg = vegLoss(regions)

1: -26.58362637301275% Vegetation Loss
2: 0.0% Vegetation Loss
3: 10.402343258404615% Vegetation Loss
4: 58.65985393888328% Vegetation Loss
5: -80.29933006837024% Vegetation Loss
6: -564.5097177138656% Vegetation Loss
7: -4.952143638584937% Vegetation Loss
8: 0.0% Vegetation Loss
9: 0.0% Vegetation Loss
10: 0.0% Vegetation Loss
11: -70.23355293384684% Vegetation Loss
12: 0.0% Vegetation Loss
13: 57.36733301065784% Vegetation Loss
14: 64.02360822487636% Vegetation Loss
15: -16.404261577648512% Vegetation Loss
16: 100.0% Vegetation Loss
17: 82.26592731200685% Vegetation Loss
18: 16.19223770034255% Vegetation Loss
19: -34.78804780037075% Vegetation Loss
20: -19.320689702281587% Vegetation Loss
21: -29.452475499031884% Vegetation Loss
22: 37.22961451686399% Vegetation Loss
23: 10.609123263857922% Vegetation Loss
24: 83.79185647682702% Vegetation Loss
25: 93.67274542472043% Vegetation Loss
26: No Vegetation at Start or End
27: 100.0% Vegetation Loss
28: 38.56722259737718% Vegetation Loss
29

In [186]:
Map

Map(bottom=1092976.0, center=[-7.54829224824849, 27.393320353717847], controls=(WidgetControl(options=['positi…

<h3> Evaluating Regions by NDMI </h3>

In [187]:
def create_ndmi_layers_modis(start, end, geometry):
    images = []
    ndmiStart = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .median() \
                .clip(geometry)
    ndmiStart2 = ndmiStart.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band6']).rename('NDMI')
    images.append((ndmiStart2, start))
    ndmiEnd = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .median() \
                .clip(geometry)
    ndmiEnd2 = ndmiEnd.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band6']).rename('NDMI')
    images.append((ndmiEnd2, end))
    return images

In [188]:
def extract_ndmi_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'NDMI': stats.get('NDMI')
    }
    
    return ee.Feature(None, properties)

In [189]:
def ndmiLoss(regionList):
    #Map = geemap.Map()
    passed_ndmi = []
    num = 1
    for region in regionList:
        imagesNDMI = ee.ImageCollection(create_ndmi_layers_modis(2000, 2020, region))
        dataNDMI = ee.FeatureCollection(imagesNDMI.map(extract_ndmi_modis))
        dfNDMI = geemap.ee_to_pandas(dataNDMI)
        loss = dfNDMI.iat[0,1]-dfNDMI.iat[1,1]
        print(str(num) + ': ' + str(loss) + ' NDMI Loss')
        num = num+1
        if loss > 0:
             passed_ndmi.append(region)
    if len(passed_ndmi) > 0:
        
        layer_ndmi = ee.Geometry.MultiPolygon(passed_ndmi)
        image = s2 \
            .filter(ee.Filter.bounds(layer_ndmi)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(layer_ndmi)
        Map.addLayer(image, rgbVisS2, 'pass ndmi')
    Map
    return passed_ndmi

In [190]:
passed_ndmi = ndmiLoss(passed_veg)

1: 0.017244721862564635 NDMI Loss
2: 0.015221081795160171 NDMI Loss
3: 0.04323306365100267 NDMI Loss
4: -0.007118734313111927 NDMI Loss
5: -0.013782842636708863 NDMI Loss
6: 0.012069184703806274 NDMI Loss
7: 0.12955915266323845 NDMI Loss
8: 0.12995933095001472 NDMI Loss
9: 0.09110831633918179 NDMI Loss
10: 0.037093876731847504 NDMI Loss
11: 0.0038574559095642252 NDMI Loss
12: 0.08921465015964233 NDMI Loss
13: 0.04408220388484482 NDMI Loss
14: 0.10175740618148538 NDMI Loss
15: 0.07200705811829107 NDMI Loss
16: 0.0072233737515542545 NDMI Loss
17: -0.0036080723778562085 NDMI Loss
18: 0.03921648620026212 NDMI Loss
19: 0.0036456714508246446 NDMI Loss
20: 0.022892197655552792 NDMI Loss
21: -0.038924305669345516 NDMI Loss
22: 0.0006124206797471082 NDMI Loss
23: -0.03116386190090424 NDMI Loss


In [191]:
Map

Map(bottom=1092976.0, center=[-7.54829224824849, 27.393320353717847], controls=(WidgetControl(options=['positi…

<h3> Evaluating Regions by NIR/SWIR2 </h3>

In [192]:
def create_nirSwir2_layers_modis(start, end, geometry):
    images = []
    nirSwir2Start = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .median() \
                .clip(geometry)
    nirSwir2Start2 = nirSwir2Start.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band7']).rename('nirSwir2')
    images.append((nirSwir2Start2, start))
    nirSwir2End = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .median() \
                .clip(geometry)
    nirSwir2End2 = nirSwir2End.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band7']).rename('nirSwir2')
    images.append((nirSwir2End2, end))
    return images

In [193]:
def extract_nirSwir2_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'nirSwir2': stats.get('nirSwir2')
    }
    
    return ee.Feature(None, properties)

In [194]:
def ns2Loss(regionList):
    passed_ns2 = []
    num = 1
    for region in regionList:
        imagesNirSwir2 = ee.ImageCollection(create_nirSwir2_layers_modis(2000, 2020, region))
        dataNirSwir2 = ee.FeatureCollection(imagesNirSwir2.map(extract_nirSwir2_modis))
        dfNirSwir2 = geemap.ee_to_pandas(dataNirSwir2)
        loss = dfNirSwir2.iat[0,1]-dfNirSwir2.iat[1,1]
        print(str(num) + ': ' + str(loss) + ' NIR/SWIR2 Loss')
        num = num+1
        if loss > 0:
            passed_ns2.append(region)
    if len(passed_ns2) > 0:
        layer_ns2 = ee.Geometry.MultiPolygon(passed_ns2)
        image = s2 \
            .filter(ee.Filter.bounds(layer_ns2)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(layer_ns2)
        Map.addLayer(image, rgbVisS2, 'pass nir/swir2')
    Map
    return passed_ns2

In [195]:
passed_ns2 = ns2Loss(passed_ndmi)

1: -0.005699903063529632 NIR/SWIR2 Loss
2: 0.024499929306567136 NIR/SWIR2 Loss
3: 0.05521097662673102 NIR/SWIR2 Loss
4: 0.005653463396842084 NIR/SWIR2 Loss
5: 0.17604630945004446 NIR/SWIR2 Loss
6: 0.2414937623333711 NIR/SWIR2 Loss
7: 0.2317196915535716 NIR/SWIR2 Loss
8: 0.14203091777506666 NIR/SWIR2 Loss
9: 0.002389426798608818 NIR/SWIR2 Loss
10: 0.10873056021271238 NIR/SWIR2 Loss
11: 0.03216808840756796 NIR/SWIR2 Loss
12: 0.16199154933059118 NIR/SWIR2 Loss
13: 0.16662272003127082 NIR/SWIR2 Loss
14: 0.10879453610138073 NIR/SWIR2 Loss
15: 0.008373168272671483 NIR/SWIR2 Loss
16: -0.03314054695118973 NIR/SWIR2 Loss
17: 0.12665095531090512 NIR/SWIR2 Loss
18: 0.020718840300245067 NIR/SWIR2 Loss


In [196]:
Map

Map(bottom=1092976.0, center=[-7.54829224824849, 27.393320353717847], controls=(WidgetControl(options=['positi…

<h3> Evaluating Regions by NIR/G </h3>

In [197]:
def create_nirG_layers_modis(start, end, geometry):
    images = []
    nirGStart = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .median() \
                .clip(geometry)
    nirGStart2 = nirGStart.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band4']).rename('nirG')
    images.append((nirGStart2, start))
    nirGEnd = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .median() \
                .clip(geometry)
    nirGEnd2 = nirGEnd.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band4']).rename('nirG')
    images.append((nirGEnd2, end))
    return images

In [198]:
def extract_nirG_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'nirG': stats.get('nirG')
    }
    
    return ee.Feature(None, properties)

In [199]:
def ngLoss(regionList):
    passed_ng = []
    num = 1
    for region in regionList:
        imagesNirG = ee.ImageCollection(create_nirG_layers_modis(2000, 2020, region))
        dataNirG = ee.FeatureCollection(imagesNirG.map(extract_nirG_modis))
        dfNirG = geemap.ee_to_pandas(dataNirG)
        loss = dfNirG.iat[0,0]-dfNirG.iat[1,0]
        print(str(num) + ': ' + str(loss) + ' NIR/G Loss')
        num = num+1
        if loss > 0.02:
            passed_ng.append(region)
    if len(passed_ng) > 0:
        layer_ng = ee.Geometry.MultiPolygon(passed_ng)
        image = s2 \
            .filter(ee.Filter.bounds(layer_ng)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(layer_ng)
        Map.addLayer(image, rgbVisS2, 'pass nir/g')
    Map
    return passed_ng

In [200]:
passed_ng = ngLoss(passed_ns2)

1: -0.0043559256442531336 NIR/G Loss
2: 0.013354291295854415 NIR/G Loss
3: -0.007642127197205029 NIR/G Loss
4: 0.06539817090284505 NIR/G Loss
5: 0.07993697681532941 NIR/G Loss
6: 0.14173912577939674 NIR/G Loss
7: 0.055720661284736295 NIR/G Loss
8: -0.01862329513723804 NIR/G Loss
9: 0.059917949498527245 NIR/G Loss
10: 0.007610462110373306 NIR/G Loss
11: 0.10376656340603008 NIR/G Loss
12: 0.03909728772008225 NIR/G Loss
13: -0.0024431271271367905 NIR/G Loss
14: 0.02096525230624091 NIR/G Loss
15: 0.002134978405744392 NIR/G Loss
16: -0.03696058343432407 NIR/G Loss


In [201]:
Map

Map(bottom=1092976.0, center=[-7.54829224824849, 27.393320353717847], controls=(WidgetControl(options=['positi…

<h3> Evaluate Regions to find Tailing Ponds </h3>

In [202]:
def ndmiGain(regionList):
    Map.centerObject(focus, 13)
    passed_ndmi_gain = []
    num = 1
    for region in regionList:
        imagesNDMI = ee.ImageCollection(create_ndmi_layers_modis(2000, 2020, region))
        dataNDMI = ee.FeatureCollection(imagesNDMI.map(extract_ndmi_modis))
        dfNDMI = geemap.ee_to_pandas(dataNDMI)
        gain = dfNDMI.iat[0,1] - dfNDMI.iat[1,1]
        print(str(num) + ': ' + str(gain) + ' NDMI Gain')
        num = num+1
        if gain < 0:
            passed_ndmi_gain.append(region)
    print(len(passed_ndmi_gain))
    if len(passed_ndmi_gain) > 0:
        layer_ndmi = ee.Geometry.MultiPolygon(passed_ndmi_gain)
        image = s2 \
            .filter(ee.Filter.bounds(layer_ndmi)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(layer_ndmi)
        Map.addLayer(image, rgbVisS2, 'pass_ndmi_gain')
    return passed_ndmi_gain

In [203]:
tailingPonds = ndmiGain(passed_veg)

1: 0.017244721862564635 NDMI Gain
2: 0.015221081795160171 NDMI Gain
3: 0.04323306365100267 NDMI Gain
4: -0.007118734313111927 NDMI Gain
5: -0.013782842636708863 NDMI Gain
6: 0.012069184703806274 NDMI Gain
7: 0.12955915266323845 NDMI Gain
8: 0.12995933095001472 NDMI Gain
9: 0.09110831633918179 NDMI Gain
10: 0.037093876731847504 NDMI Gain
11: 0.0038574559095642252 NDMI Gain
12: 0.08921465015964233 NDMI Gain
13: 0.04408220388484482 NDMI Gain
14: 0.10175740618148538 NDMI Gain
15: 0.07200705811829107 NDMI Gain
16: 0.0072233737515542545 NDMI Gain
17: -0.0036080723778562085 NDMI Gain
18: 0.03921648620026212 NDMI Gain
19: 0.0036456714508246446 NDMI Gain
20: 0.022892197655552792 NDMI Gain
21: -0.038924305669345516 NDMI Gain
22: 0.0006124206797471082 NDMI Gain
23: -0.03116386190090424 NDMI Gain
5


In [204]:
Map

Map(bottom=1092976.0, center=[-7.54829224824849, 27.393320353717847], controls=(WidgetControl(options=['positi…